## Imports & Variables

In [203]:
#Imports

import os
import time
import joblib
from tqdm import tqdm
from IPython.display import clear_output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Linear models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso

#Ensemble models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

#Other models
from sklearn.neighbors import KNeighborsRegressor

#Sklearn
from sklearn.datasets import _california_housing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV

#Settings
plt.style.use('dark_background')

In [201]:
#Variables

#Creation du dataframe avec le dataset
dataset = _california_housing.fetch_california_housing(as_frame=True)
df = dataset.frame

#Definition de la target
target_name = "MedHouseVal"
target = df[target_name]

#Colonnes exclues pour le test
columns_to_drop = [
    target_name, 
    # "Population", 
    # "AveOccup", 
    # "AveBedrms", 
    # "HouseAge", 
    # "AveRooms"
]
data = df.drop(columns=columns_to_drop)


data, target = _california_housing.fetch_california_housing(as_frame=True, return_X_y=True)
#Split
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=69)

#Modèles à tester
linear_models = [
    LinearRegression(),
    Ridge(random_state=69),
    RidgeCV(),
    Lasso(random_state=69),
]

ens_models = [
    AdaBoostRegressor(random_state=69),  
    BaggingRegressor(random_state=69, n_jobs=-1), 
    ExtraTreesRegressor(random_state=69, n_jobs=-1), 
    GradientBoostingRegressor(random_state=69), 
    RandomForestRegressor(random_state=69, n_jobs=-1), 
    HistGradientBoostingRegressor(random_state=69)
]

boost_and_bag_models = [
    AdaBoostRegressor(base_estimator=BaggingRegressor(random_state=69, n_jobs=-1)),
    AdaBoostRegressor(base_estimator=ExtraTreesRegressor(random_state=69, n_jobs=-1)),
    AdaBoostRegressor(base_estimator=GradientBoostingRegressor(random_state=69)),
    AdaBoostRegressor(base_estimator=RandomForestRegressor(random_state=69, n_jobs=-1)),
    AdaBoostRegressor(base_estimator=HistGradientBoostingRegressor(random_state=69)),

    BaggingRegressor(base_estimator=BaggingRegressor(random_state=69, n_jobs=-1)),
    BaggingRegressor(base_estimator=ExtraTreesRegressor(random_state=69, n_jobs=-1)),
    BaggingRegressor(base_estimator=GradientBoostingRegressor(random_state=69)),
    BaggingRegressor(base_estimator=RandomForestRegressor(random_state=69, n_jobs=-1)),
    BaggingRegressor(base_estimator=HistGradientBoostingRegressor(random_state=69)),
]

# AdaBoostRegressor(random_state=69, learning_rate=1,n_estimators=115, base_estimator=HistGradientBoostingRegressor(random_state=69, learning_rate=0.1, max_iter=1000, max_depth=7, warm_start=True, max_leaf_nodes=None))

## I - Début de l'analyse des données

In [ ]:
#Infos du dataset

print(dataset.DESCR)

In [ ]:
#Heatmap

data = df.drop(columns=["Latitude", "Longitude"])
plt.figure(figsize=(12,7))
sns.heatmap(data.corr(),cbar=True,annot=True,cmap='Blues')

In [ ]:
#Prix moyen en fonction de l'age des maisons

sns.lineplot(df,
                x="HouseAge", y="MedHouseVal", alpha=0.5)
plt.title("MedHouseVal based on HouseAge")

plt.show()

In [ ]:
#Prix moyen en fonction du nombre de pièces

sns.lineplot(df,
                x="AveBedrms", y="MedHouseVal", alpha=0.5)
plt.title("MedHouseVal based on AveRooms")

plt.show()

In [ ]:
#Prix moyen en fonction de la localisation géographique

sns.scatterplot(df,
                x="Longitude", y="Latitude",
                size="MedHouseVal", hue="MedHouseVal",
                palette="viridis", alpha=0.5)
plt.legend(title="MedHouseVal", bbox_to_anchor=(1.05, 1),
           loc="upper left")
plt.title("Median house value depending of\n their world location")

plt.show()

## II - Recherche du modèle adéquat

### 1) Premiers essais

In [ ]:
#RandomForestClassifier

modele_rf = RandomForestClassifier(
    # il s’agit du nombre d’arbres dans la forêt
    n_estimators=5,
    # il s’agit du critère utilisé pour construire les arbres et séparer les branches des arbres
    criterion='gini',
    # il s’agit de la profondeur maximale des arbres utilisés (le nombre de niveaux dans l’arbre de décision)
    max_depth=None,
    # il s’agit du nombre d’échantillons minimal dans une feuille pour refaire une séparation
    min_samples_split=2,
    # il s’agit du nombre d’échantillons minimal pour créer une feuille
    min_samples_leaf=1,
    # il s’agit de la fraction du nombre total d’échantillon minimal pour créer une feuille
    min_weight_fraction_leaf=0.0,
    # il s’agit du nombre maximal de feuilles
    max_leaf_nodes=None,
    # il s’agit de la baisse minimale du critère d’impureté pour faire une séparation
    min_impurity_decrease=0.0,
    # paramètre pour utiliser du bootstrap, si il est à False, le même échantillon est pris pour chaque arbre
    bootstrap=True,
    # ??
    oob_score=False,
    # nombre de traitements à effectuer en parallèle
    n_jobs=None,
    # graine aléatoire
    random_state=None,
    # ??
    verbose=0,
    # ceci permet de repartir du résultat du dernier apprentissage pour faire l’apprentissage
    warm_start=False,
    # il s’agit des poids associés à chaque classe si cela a un sens
    class_weight=None,
    # ??
    ccp_alpha=0.0,
    # si vous voulez réduire le nombre d’observations dans vos échantillons bootstrap
    max_samples=None,
)

modele_rf.fit(data, target)

In [ ]:
#KNeighborsRegressor

#Colonnes exclues pour le train
KNeighborsRegressor_data = df.drop(columns=[target_name, "AveOccup", "Population", "HouseAge", "AveBedrms" , "AveRooms"])

#Entrainement
KNeighborsRegressor_model = KNeighborsRegressor()
KNeighborsRegressor_model.fit(KNeighborsRegressor_data, target)

target_predicted = KNeighborsRegressor_model.predict(KNeighborsRegressor_data)
print(target[:5])
print(target_predicted[:5])
print(f"Number of correct prediction: "
      f"{(target[:5] == target_predicted[:5]).sum()} / 5")

### 2) Modèles linéaires

In [ ]:
#Test de tous les modèles linéaires

#Nombre de tests par modèle
attemps = 200

def TestLinearModels(models):
    
    #Initialisation de la dataframe des resultats
    result_df = pd.DataFrame(columns=["ModelName","AvgScore","MaxScore","AvgExecTime"])

    for model in models:

        #Affichage
        model_name = model.__class__.__name__
        print(f"{model_name}")

        #Initialisation des listes
        score_results = []
        time_results = []
        
        for i in tqdm(range(attemps)):

            #Demarrage du timer
            start = time.time()

            #Fit
            model.fit(data_train, target_train)

            #Calcul du score
            score = model.score(data_test, target_test)

            #Sauvegarde des resultats dans les listes
            score_results.append(score)
            time_results.append(time.time() - start)

        #Calcul des stats
        avg_score = round(np.average(score_results) * 100, 3)
        max_score = round(np.max(score_results) * 100, 3)
        avg_time = round(np.average(time_results), 3)

        #Sauvegarde des resultats dans la dataframe
        new_row = pd.DataFrame([[model_name, avg_score, max_score, avg_time]], columns=["ModelName","AvgScore","MaxScore","AvgExecTime"])
        result_df = pd.concat([result_df, new_row])

    #Affichage
    print(f"-"*49, "\nResults :")
    print(result_df.to_string(index=False))
    print(f"-"*49)

#Call de la fonction
TestLinearModels(linear_models)

In [ ]:
#Learning curves des modèles linéaires

def linear_learnings_curves(
    models,
    data,
    target
):
    plt.figure(1)
    fig, graphs = plt.subplots(1, len(models), figsize=(5* len(models), 5))
    n_jobs = -1
    points_amount = 100
    train_score_arr = []
    test_scores_arr = []

    for i in tqdm(range(len(models))):
        graphs[i].set_title(models[i].__class__.__name__)
        graphs[i].set_xlabel("Training set size")
        graphs[i].set_ylabel("Error")
        
        train_sizes, train_scores, test_scores = learning_curve(
        random_state=69,
        estimator = models[i],
        X=data,
        y=target,
        n_jobs=n_jobs,
        train_sizes=np.linspace(0.1, 1, points_amount),
        scoring="neg_mean_squared_error"
        )

        train_scores_mean = -np.mean(train_scores, axis=1)
        test_scores_mean = -np.mean(test_scores, axis=1)

        train_score_arr.append(train_scores_mean)
        test_scores_arr.append(test_scores_mean)

    for i in range(len(train_score_arr)):
        graphs[i].set_ylim(0,2)
        graphs[i].plot(train_sizes, train_score_arr[i])
        graphs[i].plot(train_sizes, test_scores_arr[i])
        graphs[i].legend(labels=["train", "test"], loc="lower right")

    return plt


#Call de la fonction
linear_learnings_curves(
    linear_models,
    data,
    target
)

plt.show()

### 3) Ensembles

In [ ]:
#Test de tous les modèles ensemblistes

#Nombre de tests par modèle
attemps = 10

def TestEnsModels(models):
    
    #Initialisation de la dataframe des resultats
    result_df = pd.DataFrame(columns=["ModelName","AvgScore","MaxScore","AvgExecTime"])

    for model in models:

        #Affichage
        model_name = model.__class__.__name__
        print(f"{model_name}")

        #Initialisation des listes
        score_results = []
        time_results = []
        
        for i in tqdm(range(attemps)):

            #Demarrage du timer
            start = time.time()

            #Fit
            model.fit(data_train, target_train)

            #Calcul du score
            score = model.score(data_test, target_test)

            #Sauvegarde des resultats dans les listes
            score_results.append(score)
            time_results.append(time.time() - start)

        #Calcul des stats
        avg_score = round(np.average(score_results) * 100, 3)
        max_score = round(np.max(score_results) * 100, 3)
        avg_time = round(np.average(time_results), 3)

        #Sauvegarde des resultats dans la dataframe
        new_row = pd.DataFrame([[model_name, avg_score, max_score, avg_time]], columns=["ModelName","AvgScore","MaxScore","AvgExecTime"])
        result_df = pd.concat([result_df, new_row])

    #Affichage
    print(f"-"*62, "\nResults :")
    print(result_df.to_string(index=False))
    print(f"-"*62)

#Call de la fonction
TestEnsModels(ens_models)

In [ ]:
#Learning curves des modèles ensemblistes

def ens_learnings_curves(
    models,
    data,
    target
):
    plt.figure(1)
    fig, graphs = plt.subplots(1, len(models), figsize=(5* len(models), 5))
    n_jobs = -1
    points_amount = 10
    train_score_arr = []
    test_scores_arr = []

    for i in tqdm(range(len(models))):
        graphs[i].set_title(models[i].__class__.__name__)
        graphs[i].set_xlabel("Training set size")
        graphs[i].set_ylabel("Error")
        
        train_sizes, train_scores, test_scores = learning_curve(
        random_state=69,
        estimator = models[i],
        X=data,
        y=target,
        n_jobs=n_jobs,
        train_sizes=np.linspace(0.1, 1, points_amount),
        scoring="neg_mean_squared_error"
        )

        train_scores_mean = -np.mean(train_scores, axis=1)
        test_scores_mean = -np.mean(test_scores, axis=1)

        train_score_arr.append(train_scores_mean)
        test_scores_arr.append(test_scores_mean)


    for i in range(len(train_score_arr)):
        graphs[i].set_ylim(0,2)
        graphs[i].plot(train_sizes, train_score_arr[i])
        graphs[i].plot(train_sizes, test_scores_arr[i])
        graphs[i].legend(labels=["train", "test"], loc="upper right")

    return plt

#Call de la fonction
ens_learnings_curves(
    ens_models,
    data,
    target
)

plt.show()

## III - Recherche des meilleurs hyperparamètres

##### Recherche des meilleurs hyperparamètres pour le HistGradientBoostingRegressor

In [190]:
#Fonction fit + score
def fit_and_score(alg):
    alg.fit(data_train, target_train)
    print("R2 : "+str(alg.score(data_test, target_test)*100))
    return alg.score(data_test, target_test)*100

In [162]:
#Modèle de base
model = HistGradientBoostingRegressor(
    random_state=69,
)

fit_and_score(model)

82.85875163749014

In [163]:
#Recherche "max_iter"
model = HistGradientBoostingRegressor(
    random_state=69,
    min_samples_leaf=20,
    max_depth=12,
    learning_rate=0.1,
)

parameters = {
    'max_iter':range(10,501,10),
}

clf = GridSearchCV(estimator=model, param_grid=parameters)

clf.fit(data_train, target_train)
print(clf.best_params_)
print(clf.score(data_test, target_test)*100)

{'max_iter': 280}
83.91491564971824


In [165]:
#Recherche "min_samples_leaf" & "max_depth"
model = HistGradientBoostingRegressor(
    random_state=69,
    max_iter=280,
    learning_rate=0.1,
)

parameters = {
    'min_samples_leaf':range(10, 121, 10),
    'max_depth':range(10,21,2),
}

clf = GridSearchCV(estimator=model, param_grid=parameters)

clf.fit(data_train, target_train)
print(clf.best_params_)
print(clf.score(data_test, target_test)*100)

{'max_depth': 18, 'min_samples_leaf': 40}
84.069723920638


In [185]:
#Final HistGradientBoostingRegressor parameters
model = HistGradientBoostingRegressor(
    random_state=69,
    max_depth=18,
    min_samples_leaf=40,
    max_iter=280,
    learning_rate=0.1,
    warm_start=True
)

fit_and_score(model)

r = np.absolute(model.predict(data) - target)*100

print(str(np.mean(r)) + " +- " + str(np.std(r)))

25.067762567270965 +- 28.42309588816349


##### Recherche des meilleurs hyperparamètres pour le AdaBoostRegressor

In [186]:
#Modèle de base
m = AdaBoostRegressor(random_state=69, base_estimator=model)
fit_and_score(m)

84.58150365307209

In [188]:
#Recherche "n_estimators" & "learning_rate"
m = AdaBoostRegressor(random_state=69,
                      base_estimator=model,
                      learning_rate=0.1)

parameters = {
    'n_estimators':range(40, 51, 5),
    'learning_rate':[0.1, 1],
}

clf = GridSearchCV(estimator=m, param_grid=parameters)

clf.fit(data_train, target_train)
print(clf.best_params_)
print(clf.score(data_test, target_test)*100)

{'learning_rate': 0.1, 'n_estimators': 40}
84.81381972890627


In [205]:
#Final Results Ada(Hist())
final_model = AdaBoostRegressor(random_state=69, learning_rate=1,n_estimators=115, base_estimator=HistGradientBoostingRegressor(learning_rate=0.1, max_iter=1000, max_depth=7, warm_start=True, max_leaf_nodes=60, random_state=69,  max_bins=255))

fit_and_score(final_model)

r = np.absolute(final_model.predict(data) - target)*100

print(str(np.mean(r)) + " +- " + str(np.std(r)))

R2 : 85.1542366577762
11.952681698961994 +- 17.70402581658711


## IV - Sauvegarde de l'IA

In [207]:
#Sauvegarde de l'IA

def AI_train_and_save(model):
    
    #Initialisation de la dataframe des resultats
    result_df = pd.DataFrame(columns=["ModelName","Score"])

    #Affichage
    model_name = model.__class__.__name__ + "(" +  model.base_estimator.__class__.__name__ + "())"

    #Fit
    model.fit(data_train, target_train)

    #Calcul du score
    score = model.score(data_test, target_test)*100

    #Sauvegarde des resultats dans la dataframe
    new_row = pd.DataFrame([[model_name, score]], columns=["ModelName","Score"])
    result_df = pd.concat([result_df, new_row])

    #Affichage du score
    print(result_df)

    #Sauvegarde de l'IA
    with open(f"{score}.joblib", "wb") as fo:
        joblib.dump(model, fo)

#Call de la fonction
AI_train_and_save(final_model)

                                           ModelName     Score
0  AdaBoostRegressor(HistGradientBoostingRegresso...  0.851542


## V - Test de l'IA

In [211]:
#Test

#Chargement de l'IA sauvegardée
with open("0.8515423665777619.joblib", "rb") as fo:
    cf = joblib.load(fo)

#Predict
predicted = cf.predict(data)
d = {'Predicted': predicted, 'Original': target, 'Ecart': np.absolute(predicted-target)*100000}

#Stockage et affichage du resultat
supa_resulto_dataframu = pd.DataFrame(data=d)
print(supa_resulto_dataframu)

#Affichage des stats
print(np.min(supa_resulto_dataframu["Ecart"]))
print(np.average(supa_resulto_dataframu["Ecart"]))
print(np.median(supa_resulto_dataframu["Ecart"]))

       Predicted  Original          Ecart
0       4.432357     4.526    9364.269210
1       3.630886     3.585    4588.646308
2       4.582870     3.521  106187.002778
3       3.466137     3.413    5313.689268
4       3.315778     3.422   10622.246697
...          ...       ...            ...
20635   0.768587     0.781    1241.282860
20636   0.828364     0.771    5736.406130
20637   0.897097     0.923    2590.302024
20638   0.898776     0.847    5177.629022
20639   0.874969     0.894    1903.095152

[20640 rows x 3 columns]
0.5284826749663019
11952.681698961995
8594.969820317034
